In [1]:
!pip install mlxtend --user
!pip install lightgbm --user
!pip install xgboost --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.9 MB/s eta 0:00:00


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy as sp
import math
import random
import seaborn as sn
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Importing the Datasets

## Model Performance Metrics

In [2]:
def rmse(y, y_pred):
    return np.sqrt(np.mean((y - y_pred)**2))

def mae(y, y_pred):
    return np.mean(np.absolute(np.subtract(y, y_pred)))

def r2(y, y_pred):
    return 1 - np.sum((y - y_pred)**2)/np.sum((y - np.mean(y))**2)

def mape(y, y_pred):
    return (np.mean(np.divide(np.absolute(np.subtract(y,y_pred)), y))*100)

def errors(y, y_pred):
    return np.absolute(y - y_pred)

In [3]:
df_winter = pd.read_csv('/mnt/ETL/Datasets/yield_prediction_dataset.csv').drop(columns = ['Unnamed: 0'])

## Using a Class

In [4]:
class PredictorWithScalars:
    
    def __init__(self, predictor, X_scalar, X_transformer, y_scalar, y_transformer):
        self.predictor = predictor
        self.X_scalar = X_scalar
        self.X_transformer = X_transformer
        self.y_scalar = y_scalar
        self.y_transformer = y_transformer
    
    def transform_X(self, X):
        transformed_X = self.X_transformer.transform(self.X_scalar.transform(X))
        return transformed_X
    
    def transform_y(self, y):
        transformed_y = self.y_transformer.transform(self.y_scalar.transform(y))
        return transformed_y

    def predict(self, X):
        transformed_X = self.X_transformer.transform(self.X_scalar.transform(X))
        transformed_y_preds = self.predictor.predict(transformed_X).reshape(-1,1)
        inverse_transformed_y_preds = self.y_scalar.inverse_transform(self.y_transformer.inverse_transform(transformed_y_preds))
        return inverse_transformed_y_preds

In [5]:
def build_model(dataset, outlier_removal_lower_percentile, seed):
    
    dataset = dataset[(dataset['Yield_kg_per_Ha'] > dataset['Yield_kg_per_Ha'].quantile(outlier_removal_lower_percentile)) \
                                                                   & (dataset['Yield_kg_per_Ha'] < dataset['Yield_kg_per_Ha'].quantile(1-outlier_removal_lower_percentile))].reset_index(drop = True)

    train_Yield_kg_per_ha, test_Yield_kg_per_ha = train_test_split(dataset, test_size = 0.18, random_state = seed)

    X_train_Yield_kg_per_ha = train_Yield_kg_per_ha.drop(columns = ['Yield_kg_per_Ha'])
    X_test_Yield_kg_per_ha = test_Yield_kg_per_ha.drop(columns = ['Yield_kg_per_Ha'])

    y_train_Yield_kg_per_ha = np.array(train_Yield_kg_per_ha['Yield_kg_per_Ha']).reshape(-1, 1)
    y_test_Yield_kg_per_ha = np.array(test_Yield_kg_per_ha['Yield_kg_per_Ha']).reshape(-1, 1)
    mean_Yield_kg_per_ha = np.mean(y_train_Yield_kg_per_ha)

    standard_scaler_X = StandardScaler().fit(X_train_Yield_kg_per_ha)
    power_transform_X = PowerTransformer(standardize = True).fit(standard_scaler_X.transform(X_train_Yield_kg_per_ha))

    standard_scaler_y = StandardScaler().fit(y_train_Yield_kg_per_ha)
    power_transform_y = PowerTransformer(standardize = True).fit(standard_scaler_y.transform(y_train_Yield_kg_per_ha))

    xgboost = XGBRegressor(learning_rate = 0.005, n_estimators = 10000, max_depth = 3, min_child_weight = 0, gamma = 0, subsample = 0.7, \
                                              colsample_bytree = 0.7, objective = 'reg:linear', nthread = -1, scale_pos_weight = 1, seed = 27, reg_alpha = 0.00006)

    predictorScaler = PredictorWithScalars(xgboost, standard_scaler_X, power_transform_X, standard_scaler_y, power_transform_y)
    predictorScaler.predictor = predictorScaler.predictor.fit(predictorScaler.transform_X(X_train_Yield_kg_per_ha), predictorScaler.transform_y(y_train_Yield_kg_per_ha))
    
    return predictorScaler

In [6]:
def eval_model(df_winter_combined_parents_final_test_set, model_build_dict):
    
    y_preds = np.zeros((len(df_winter_combined_parents_final_test_set), len(model_build_dict)))
    
    for i in range(len(model_build_dict)):

        X_df_winter_combined_parents_final_test_set = df_winter_combined_parents_final_test_set.drop(columns = ['Yield_kg_per_Ha'])
        y_df_winter_combined_parents_final_test_set = np.array(df_winter_combined_parents_final_test_set['Yield_kg_per_Ha']).reshape(-1,1)
        y_preds[:,i] = model_build_dict.get(i).predict(X_df_winter_combined_parents_final_test_set)[:,0]
        
    final_preds = np.mean(y_preds, axis = 1).reshape(-1,1)
    mean_abs_error = mean_absolute_error(y_df_winter_combined_parents_final_test_set, final_preds)
    root_mean_sq_error = np.sqrt(mean_squared_error(y_df_winter_combined_parents_final_test_set, final_preds))
    r_sq = r2_score(y_df_winter_combined_parents_final_test_set, final_preds)
    
    return mean_abs_error, root_mean_sq_error, r_sq

In [ ]:
df_winter_combined_parents = pd.read_csv('/mnt/ETL/Datasets/yield_prediction_dataset.csv').drop(columns = ['Unnamed: 0'])

num_seeds_list = [5, 5, 5, 5, 5]

random_state_list = random.sample(range(1000), len(num_seeds_list))

results_collection_df = pd.DataFrame(columns = ['num_models', 'mae', 'rmse', 'r_squared'])

for i,num_seeds in enumerate(num_seeds_list):
    
    df_winter_combined_parents_temp = df_winter_combined_parents.sample(frac = 1, random_state = random_state_list[i]).reset_index(drop = True)
    df_winter_combined_parents_final_test_set = df_winter_combined_parents_temp[:57]
    df_winter_combined_parents_temp = df_winter_combined_parents_temp[57:].reset_index(drop = True)
    
    seed_list = random.sample(range(1000), num_seeds)
    
    model_build_dict = {}
    
    for i,seed in enumerate(seed_list):
        model_build_dict.update({i: build_model(df_winter_combined_parents_temp, 0, seed)})
    
    eval_model_temp = eval_model(df_winter_combined_parents_final_test_set, model_build_dict)
    row = pd.DataFrame({'num_models': [num_seeds], 'mae': [eval_model_temp[0]], 'rmse': [eval_model_temp[1]], 'r_squared': [eval_model_temp[2]]})
    
    results_collection_df = pd.concat([results_collection_df, row], axis = 0)

print(display(results_collection_df))

[16:15:45] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:16:10] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:16:34] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:16:59] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:17:23] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:17:49] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:13] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:37] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:19:05] WARNING: ../src/objective/regression_

In [ ]:
df_winter_combined_parents = pd.read_csv('/mnt/ETL/Datasets/yield_prediction_dataset.csv').drop(columns = ['Unnamed: 0'])

In [ ]:
class PredictorWithScalars:
    
    def __init__(self, predictor, X_scalar, X_transformer, y_scalar, y_transformer):
        self.predictor = predictor
        self.X_scalar = X_scalar
        self.X_transformer = X_transformer
        self.y_scalar = y_scalar
        self.y_transformer = y_transformer
    
    def transform_X(self, X):
        transformed_X = self.X_transformer.transform(self.X_scalar.transform(X))
        return transformed_X
    
    def transform_y(self, y):
        transformed_y = self.y_transformer.transform(self.y_scalar.transform(y))
        return transformed_y

    def predict(self, X):
        transformed_X = self.X_transformer.transform(self.X_scalar.transform(X))
        transformed_y_preds = self.predictor.predict(transformed_X).reshape(-1,1)
        inverse_transformed_y_preds = self.y_scalar.inverse_transform(self.y_transformer.inverse_transform(transformed_y_preds))
        return inverse_transformed_y_preds

In [ ]:
def build_model(dataset, outlier_removal_lower_percentile, seed):
    
    dataset = dataset[(dataset['Yield_kg_per_Ha'] > dataset['Yield_kg_per_Ha'].quantile(outlier_removal_lower_percentile)) \
                                                                   & (dataset['Yield_kg_per_Ha'] < dataset['Yield_kg_per_Ha'].quantile(1-outlier_removal_lower_percentile))].reset_index(drop = True)

    train_Yield_kg_per_ha, test_Yield_kg_per_ha = train_test_split(dataset, test_size = 0.18, random_state = seed)

    X_train_Yield_kg_per_ha = train_Yield_kg_per_ha.drop(columns = ['Yield_kg_per_Ha'])
    X_test_Yield_kg_per_ha = test_Yield_kg_per_ha.drop(columns = ['Yield_kg_per_Ha'])

    y_train_Yield_kg_per_ha = np.array(train_Yield_kg_per_ha['Yield_kg_per_Ha']).reshape(-1, 1)
    y_test_Yield_kg_per_ha = np.array(test_Yield_kg_per_ha['Yield_kg_per_Ha']).reshape(-1, 1)
    mean_Yield_kg_per_ha = np.mean(y_train_Yield_kg_per_ha)

    standard_scaler_X = StandardScaler().fit(X_train_Yield_kg_per_ha)
    power_transform_X = PowerTransformer(standardize = True).fit(standard_scaler_X.transform(X_train_Yield_kg_per_ha))

    standard_scaler_y = StandardScaler().fit(y_train_Yield_kg_per_ha)
    power_transform_y = PowerTransformer(standardize = True).fit(standard_scaler_y.transform(y_train_Yield_kg_per_ha))

    xgboost = XGBRegressor(learning_rate = 0.005, n_estimators = 10000, max_depth = 3, min_child_weight = 0, gamma = 0, subsample = 0.7, \
                                              colsample_bytree = 0.7, objective = 'reg:linear', nthread = -1, scale_pos_weight = 1, seed = 27, reg_alpha = 0.00006)

    predictorScaler = PredictorWithScalars(xgboost, standard_scaler_X, power_transform_X, standard_scaler_y, power_transform_y)
    predictorScaler.predictor = predictorScaler.predictor.fit(predictorScaler.transform_X(X_train_Yield_kg_per_ha), predictorScaler.transform_y(y_train_Yield_kg_per_ha))
    
    return predictorScaler

In [ ]:
import pickle
# from sklearn.externals import joblib

random.seed(10)
random_state_list1 = random.sample(range(1000), 5)

random.seed(11)
random_state_list2 = random.sample(range(1000), 5)

pickle.dump(df_winter_combined_parents, open('/mnt/ETL/Datasets/yield_prediction_dataset.csv', 'wb'))

df_winter_combined_parents_temp = df_winter_combined_parents.sample(frac = 1, random_state = random_state_list1[0]).reset_index(drop = True)
df_winter_combined_parents_final_test_set = df_winter_combined_parents_temp[:57]
df_winter_combined_parents_temp = df_winter_combined_parents_temp[57:].reset_index(drop = True)

predictor_scaler1 = build_model(df_winter_combined_parents_temp, 0, random_state_list2[0])
pickle.dump(predictor_scaler1.X_scalar, open('/mnt/ETL/pkl_predictors_scalars_transformers/X_Scalar_1_yield.pkl', 'wb'))
pickle.dump(predictor_scaler1.y_scalar, open('/mnt/ETL/pkl_predictors_scalars_transformers/y_Scalar_1_yield.pkl', 'wb'))
pickle.dump(predictor_scaler1.X_transformer, open('/mnt/ETL/pkl_predictors_scalars_transformers/X_Transformer_1_yield.pkl', 'wb'))
pickle.dump(predictor_scaler1.y_transformer, open('/mnt/ETL/pkl_predictors_scalars_transformers/y_transformer_1_yield.pkl', "wb"))
pickle.dump(predictor_scaler1.predictor, open("/mnt/ETL/pkl_predictors_scalars_transformers/predictor_111_yield.pkl", "wb"))

df_winter_combined_parents_temp = df_winter_combined_parents.sample(frac = 1, random_state = random_state_list1[0]).reset_index(drop = True)
df_winter_combined_parents_final_test_set = df_winter_combined_parents_temp[:57]
df_winter_combined_parents_temp = df_winter_combined_parents_temp[57:].reset_index(drop = True)
predictor_scaler2 = build_model(df_winter_combined_parents_temp, 0, random_state_list2[1])
pickle.dump(predictor_scaler2.X_scalar, open('/mnt/ETL/pkl_predictors_scalars_transformers/X_Scalar_2_yield.pkl', 'wb'))
pickle.dump(predictor_scaler2.y_scalar, open('/mnt/ETL/pkl_predictors_scalars_transformers/y_Scalar_2_yield.pkl', 'wb'))
pickle.dump(predictor_scaler2.X_transformer, open('/mnt/ETL/pkl_predictors_scalars_transformers/X_Transformer_2_yield.pkl', 'wb'))
pickle.dump(predictor_scaler2.y_transformer, open('/mnt/ETL/pkl_predictors_scalars_transformers/y_transformer_2_yield.pkl', "wb"))
pickle.dump(predictor_scaler2.predictor, open("/mnt/ETL/pkl_predictors_scalars_transformers/predictor_222_yield.pkl", "wb"))

df_winter_combined_parents_temp = df_winter_combined_parents.sample(frac = 1, random_state = random_state_list1[0]).reset_index(drop = True)
df_winter_combined_parents_final_test_set = df_winter_combined_parents_temp[:57]
df_winter_combined_parents_temp = df_winter_combined_parents_temp[57:].reset_index(drop = True)
predictor_scaler3 = build_model(df_winter_combined_parents_temp, 0, random_state_list2[2])
pickle.dump(predictor_scaler3.X_scalar, open('/mnt/ETL/pkl_predictors_scalars_transformers/X_Scalar_3_yield.pkl', 'wb'))
pickle.dump(predictor_scaler3.y_scalar, open('/mnt/ETL/pkl_predictors_scalars_transformers/y_Scalar_3_yield.pkl', 'wb'))
pickle.dump(predictor_scaler3.X_transformer, open('/mnt/ETL/pkl_predictors_scalars_transformers/X_Transformer_3_yield.pkl', 'wb'))
pickle.dump(predictor_scaler3.y_transformer, open('/mnt/ETL/pkl_predictors_scalars_transformers/y_transformer_3_yield.pkl', "wb"))
pickle.dump(predictor_scaler3.predictor, open("/mnt/ETL/pkl_predictors_scalars_transformers/predictor_333_yield.pkl", "wb"))

df_winter_combined_parents_temp = df_winter_combined_parents.sample(frac = 1, random_state = random_state_list1[0]).reset_index(drop = True)
df_winter_combined_parents_final_test_set = df_winter_combined_parents_temp[:57]
df_winter_combined_parents_temp = df_winter_combined_parents_temp[57:].reset_index(drop = True)
predictor_scaler4 = build_model(df_winter_combined_parents_temp, 0, random_state_list2[3])
pickle.dump(predictor_scaler4.X_scalar, open('/mnt/ETL/pkl_predictors_scalars_transformers/X_Scalar_4_yield.pkl', 'wb'))
pickle.dump(predictor_scaler4.y_scalar, open('/mnt/ETL/pkl_predictors_scalars_transformers/y_Scalar_4_yield.pkl', 'wb'))
pickle.dump(predictor_scaler4.X_transformer, open('/mnt/ETL/pkl_predictors_scalars_transformers/X_Transformer_4_yield.pkl', 'wb'))
pickle.dump(predictor_scaler4.y_transformer, open('/mnt/ETL/pkl_predictors_scalars_transformers/y_transformer_4_yield.pkl', "wb"))
pickle.dump(predictor_scaler4.predictor, open("/mnt/ETL/pkl_predictors_scalars_transformers/predictor_444_yield.pkl", "wb"))

df_winter_combined_parents_temp = df_winter_combined_parents.sample(frac = 1, random_state = random_state_list1[0]).reset_index(drop = True)
df_winter_combined_parents_final_test_set = df_winter_combined_parents_temp[:57]
df_winter_combined_parents_temp = df_winter_combined_parents_temp[57:].reset_index(drop = True)
predictor_scaler5 = build_model(df_winter_combined_parents_temp, 0, random_state_list2[4])
pickle.dump(predictor_scaler5.X_scalar, open('/mnt/ETL/pkl_predictors_scalars_transformers/X_Scalar_5_yield.pkl', 'wb'))
pickle.dump(predictor_scaler5.y_scalar, open('/mnt/ETL/pkl_predictors_scalars_transformers/y_Scalar_5_yield.pkl', 'wb'))
pickle.dump(predictor_scaler5.X_transformer, open('/mnt/ETL/pkl_predictors_scalars_transformers/X_Transformer_5_yield.pkl', 'wb'))
pickle.dump(predictor_scaler5.y_transformer, open('/mnt/ETL/pkl_predictors_scalars_transformers/y_transformer_5_yield.pkl', "wb"))
pickle.dump(predictor_scaler5.predictor, open("/mnt/ETL/pkl_predictors_scalars_transformers/predictor_555_yield.pkl", "wb"))